# 時期ごとの極性・共起変遷を追う

## 必要なもの
* ID, POPLATION_ID, PERIOD_IDのセット
* local_stopwordsのセット
* 分析したキーワードをKeywords/Timelapse/ID-from-POPULATION_ID.txtにスペース区切りで保存
* get_time_index()の書き換え

In [31]:
ID = "pikachu"

POPULATION_ID = "pokemon"

PERIOD_ID = "year"

#分析対象の単語ファイル
keywords_file = "Keywords/Timelapse/%s-from-%s.txt"%(ID, POPULATION_ID)

#ストップワード抜きの形態素解析・極性分析完了したデータ
without_stopwords_text_file = "Progresses/NonStopword/%s-from-%s.txt"%(ID, POPULATION_ID)

##TFIDFモデル
tfidf_model_file = "Models/Tfidf/%s-from-%s.model"%(ID, POPULATION_ID)

#保存するファイル
sentiment_timelapse_file = "Sentiment/Timelapse/%s-from-%s-by-%s.txt"%(ID, POPULATION_ID, PERIOD_ID)

#内部的に品詞を区別する区切り文字
TOKEN_DIVIDER = "<334>"

In [32]:
#共起キーワードのストップワード（ここだけ）
local_stopwords = ["br", "ピカチュウ", "無い", "良い", "ポケモン"]

## 

## 与えられた時期にインデックスを割り振る
分けたい時期にあわせて書き換える

In [33]:
from datetime import datetime

In [34]:
#これを書き換える
#万が一 -1 が出てしまうとエラーが出てしまうので注意（上限はない）
def get_time_index(time: datetime):
    #2000年から一年ごとに割り振る
    if time == None:
        return None

    years_passed = time.year - 2000
    
    if years_passed >= 0:
        return years_passed
    else:
        return 0

#保持しているデータの形式からdatetime型に変換
def convert_time(text: str):
    if (text == None) | (text == "None"):
        return None
    else:
        return datetime.strptime(text, "%Y/%m/%d|%H/%M/%S")

#最大インデックスを算出
MAX_TIME_INDEX = get_time_index(datetime.now())

## データ読み込み

In [35]:
import pandas as pd
import numpy as np

### 処理済みデータ

In [36]:
df_main = pd.read_table(without_stopwords_text_file)

df_main.head(2)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment
0,2016/02/23|21/35/00,60432,7,初ポケモンがピカチュウバージョンでひいこら言いながら進めてたこと ピカチュウが主力なの...,*<334>初<334>ポケモン<334>が<334>ピカチュウ<334>バージョン-ver...,初<334>ポケモン<334>ピカチュウ<334>バージョン-version<334>事<3...,neg
1,2008/02/25|00/58/00,26640,154,ダークシンジとピカチュウが来たらいよいよもって 懐かしいコテ勢揃いだな,*<334>ダーク-外国<334>シンジ<334>と<334>ピカチュウ<334>が<334...,ダーク-外国<334>シンジ<334>ピカチュウ<334>懐かしい<334>小手<334>勢揃い,pos


### 本ノート用キーワード

In [37]:
with open(keywords_file, "r", encoding="utf-8") as f:
    keywords = f.read().split()

keywords

['サトシ', '可愛い', 'ライチュウ', 'アニメ-animation', '進化', 'ポッチャマ', '人気', '強い']

### 母集団からのTFIDFモデルをサルベージ

In [38]:
import pickle
from copy import deepcopy

def tokenize(text):
    return text.split(TOKEN_DIVIDER)

with open(tfidf_model_file, "rb") as f:
    tfidf_model = pickle.load(f)

#対象（名詞・形容詞）を見せる用
tfidf_model_showing = deepcopy(tfidf_model)
tfidf_model_showing.ngram = (1,1)
tfidf_model_showing.tokenizer = tokenize

## 分析

### 各テキストに時期インデックスを割り振る

In [39]:
indices = []

for _, row in df_main.iterrows():
    indices.append(get_time_index(convert_time(row["date"])))

df_main["time_index"] = indices

df_main.head(3)

,date,link_index,number,content,disassembled_all,disassembled_target,Sentiment,time_index
0,2016/02/23|21/35/00,60432,7,初ポケモンがピカチュウバージョンでひいこら言いながら進めてたこと ピカチュウが主力なの...,*<334>初<334>ポケモン<334>が<334>ピカチュウ<334>バージョン-ver...,初<334>ポケモン<334>ピカチュウ<334>バージョン-version<334>事<3...,neg,16.0
1,2008/02/25|00/58/00,26640,154,ダークシンジとピカチュウが来たらいよいよもって 懐かしいコテ勢揃いだな,*<334>ダーク-外国<334>シンジ<334>と<334>ピカチュウ<334>が<334...,ダーク-外国<334>シンジ<334>ピカチュウ<334>懐かしい<334>小手<334>勢揃い,pos,8.0
2,2008/02/25|01/00/00,26640,159,>>152 ハングル文字ですね 日本版と通信できるのでしょうか >>154 シンジ...,*<334>＞<334>＞<334>152<334>ハングル<334>文字<334>です<3...,152<334>ハングル<334>文字<334>日本<334>版<334>通信<334>15...,neu,8.0


### キーワードごとに分析

In [40]:
from IPython.display import display

#キーワードごとに
for keyword in keywords:
    scores = []
    coocs = []
    counts = []

    #時期インデックスごとに
    for time_index in range(MAX_TIME_INDEX):
        #極性スコア計算
        score = 0
        cnt = 0
        texts = []
        for _, row in df_main[df_main["time_index"] == time_index].iterrows():
            if keyword in row["disassembled_all"].split(TOKEN_DIVIDER):
                cnt += 1
                if row["Sentiment"] == "pos":
                    score += 1
                elif row["Sentiment"] == "neg":
                    score -= 1
                texts.append(row["disassembled_target"])
        if cnt != 0:
            score /= cnt
        else:
            score = 0
        
        ##共起
        #TFIDF計算
        if cnt != 0:
            tfidf_spycy = tfidf_model_showing.transform(texts)
            tfidf_df = pd.DataFrame(tfidf_spycy.toarray(), columns=tfidf_model_showing.get_feature_names_out())
            tfidf_mean = tfidf_df.mean(axis=0)

            #単語名と結び付け
            tfidf_mean = tfidf_mean.sort_values(ascending=False)

            #ストップワードの削除
            for word in local_stopwords:
                try:
                    tfidf_mean = tfidf_mean.drop(word)
                except:
                    pass

            if len(tfidf_mean) > 1:
                cooc = ", ".join(tfidf_mean[1:min(6, len(tfidf_mean))].index)
            else:
                cooc = ""
        else:
            cooc = ""
        
        scores.append(score)
        coocs.append(cooc)
        counts.append(cnt)

    df_keyword = pd.DataFrame(list(zip(counts, scores, coocs)), columns=["Count", "Score", "CoOccure"])

    print("\n"+keyword)
    display(df_keyword.style.bar(subset=["Score"], align='zero', color=['#d65f5f', '#5fba7d'], vmax=1, vmin=-1))


サトシ


,Count,Score,CoOccure
0,0,0.000000,
1,0,0.000000,
2,29,-0.068966,"レオン, ラティアス, ヒロシ, ﾊｧﾊｧ, д"
3,46,0.043478,"シーン-scene, ハルカ, ロケット-rocket, マサト, 突撃"
4,169,0.207101,"ハルカ, タケシ, 進化, マサト, 話"
5,119,-0.092437,"ハルカ, タケシ, マサト, 事, 主人"
6,281,0.135231,"光, タケシ, 事, ロケット-rocket, 話"
7,326,0.064417,"光, タケシ, 事, ロケット-rocket, 時"
8,241,0.024896,"光, ポッチャマ, 事, 今, 降板"
9,158,-0.056962,"光, タケシ, 主人, 事, 霞"



可愛い


,Count,Score,CoOccure
0,0,0.000000,
1,5,0.200000,"可愛い, ソーナンス, 人, 少ない, アニメ-animation"
2,16,0.125000,"コンテスト-contest, ピカチュウ 可愛い, ベイ-bay, ベイ-bay リーフ-leaf, ラティアス"
3,94,0.021277,"ピカチュウ 可愛い, 方, 可愛い ポケモン, 優遇, ﾊｧﾊｧ"
4,92,0.141304,"ピカチュウ 可愛い, 可愛い ピカチュウ, ライチュウ, 進化, 可愛い 無い"
5,78,0.435897,"ピカチュウ 可愛い, 可愛い ピカチュウ, 方, 今, プラマイ-plus minus"
6,95,0.231579,"ピカチュウ 可愛い, 可愛い ピカチュウ, 方, 事, ライチュウ"
7,107,0.214953,"ピカチュウ 可愛い, ライチュウ, 人気, 可愛い ピカチュウ, 可愛い 無い"
8,128,0.304688,"ピカチュウ 可愛い, ポッチャマ, サトシ, 可愛い ポケモン, 今"
9,67,0.223881,"ピカチュウ 可愛い, 可愛い 無い, サトシ, 可愛い ピカチュウ, 廃人"



ライチュウ


...

,Count,Score,CoOccure
0,0,0.000000,
1,0,0.000000,
2,0,0.000000,
3,0,0.000000,
4,0,0.000000,
5,0,0.000000,
6,43,0.139535,"月 28, ９ 月, 監督, 放送, 光"
7,268,0.007463,"人気, ツー-two, ミュウ, ミュウ ツー-two, ナエトル"
8,175,0.080000,"光, プッシュ-push, サトシ, 人気, ルカ リオ"
9,67,0.000000,"光, サトシ, 人気, ハルカ, 主人"



人気


,Count,Score,CoOccure
0,0,0.000000,
1,2,0.500000,"ゲーム-game, 人気, 軍, ゲーム-game ボーイ-boy, 戦争"
2,10,0.100000,"cgi, ピカチュウ 人気, 人気 ポケモン, スレ-thread, 2 ch"
3,19,-0.105263,"ピカチュウ 人気, 今 ピカチュウ, 視聴 率, 今, 視聴"
4,45,0.022222,"ライチュウ, アニメ-animation, ピカチュウ 人気, 人気 無い, 子供"
5,30,-0.066667,"ピカチュウ 人気, 人気 無い, 投票, 人気 ポケモン, ゲーム-game"
6,43,0.023256,"ピカチュウ 人気, 人気 ポケモン, 人気 無い, ハム太郎, 野郎"
7,148,-0.027027,"ポッチャマ, ピカチュウ 人気, 人気 無い, ミュウ ツー-two, ミュウ"
8,70,0.071429,"ピカチュウ 人気, ポッチャマ, 人気 ポケモン, 光, 世界"
9,38,-0.210526,"ピカチュウ 人気, サトシ, ポッチャマ, 人気 ポケモン, 光"



強い


,Count,Score,CoOccure
0,0,0.000000,
1,0,0.000000,
2,9,0.333333,"強い 無い, カイ オーガ, オーガ, グラードン, カイ"
3,15,0.133333,"強い 無い, 可愛い, 種, 育て, 319"
4,52,0.019231,"進化, サトシ, ライチュウ, 弱い, 技"
5,43,-0.046512,"電気, サトシ, 強い ポケモン, 普通, 最強"
6,85,-0.023529,"サトシ, lv, アニメ-animation, 事, 電気"
7,91,-0.219780,"ミュウ ツー-two, ツー-two, ミュウ, サトシ, 人気"
8,127,0.015748,"欲, 支配, サトシ, コネ-connection, 厨"
9,63,0.047619,"事, ロゼ, ロゼ リア-rear, リア-rear, レッド-red"
